In [ ]:
import strawberry
import requests
import uvicorn
from strawberry.fastapi import GraphQLRouter
from fastapi import FastAPI
import asyncio
import os

# Replace these with your actual details
LLM_API_URL = "https://api.openai.com/v1/chat/completions" # or your specific endpoint
API_KEY = os.getenv("OPENAI_API_KEY")  # Ensure your API key is set in environment variables

@strawberry.type
class LLMResponse:
    text: str

@strawberry.type
class Query:
    @strawberry.field
    def ask_ai(self, prompt: str) -> LLMResponse:
        # 1. Prepare the REST request
        headers = {
            "Authorization": f"Bearer {API_KEY}",
            "Content-Type": "application/json"
        }
        
        # This payload structure depends on your specific LLM (this is OpenAI style)
        payload = {
            "model": "gpt-3.5-turbo",
            "messages": [{"role": "user", "content": prompt}]
        }

        # 2. Call the REST LLM
        response = requests.post(LLM_API_URL, json=payload, headers=headers)
        
        if response.status_code == 200:
            result = response.json()
            # 3. Extract the text (adjust key names based on your API)
            ai_text = result['choices'][0]['message']['content']
            return LLMResponse(text=ai_text)
        else:
            return LLMResponse(text=f"Error: {response.text}")

# --- Server Setup ---
schema = strawberry.Schema(query=Query)
graphql_app = GraphQLRouter(schema)
app = FastAPI()
app.include_router(graphql_app, prefix="/graphql")

# 3. Start the server
if __name__ == "__main__":
    port = int(os.environ.get("CDSW_APP_PORT", 5004))
    config = uvicorn.Config(app, host="127.0.0.1", port=port, log_level="info", workers=1)
    server = uvicorn.Server(config)

    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            import nest_asyncio
            nest_asyncio.apply()
            loop.create_task(server.serve())
        else:
            loop.run_until_complete(server.serve())
    except KeyboardInterrupt:
        print("Server stopped manually")

INFO:     Started server process [9320]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:5004 (Press CTRL+C to quit)


INFO:     127.0.0.1:63324 - "GET /graphql HTTP/1.1" 200 OK
INFO:     127.0.0.1:54287 - "POST /graphql HTTP/1.1" 200 OK
INFO:     127.0.0.1:54287 - "POST /graphql HTTP/1.1" 200 OK
